In [4]:
params = {
    "input_files": {
        "praise_data": "./praise.csv",
        "rewardboard_list": "./rewardboard.csv"
      },
      "total_tokens_allocated": 1000,
      "payout_token": {
        "token_name": "wxDAI",
        "token_address": "0xe91D153E0b41518A2Ce8Dd3D7944Fa863463a97d"
      },
      "token_reward_percentages": {
        "contributor_rewards": 0.871067,
        "quantifier_rewards": 0.090324,
        "rewardboard_rewards": 0.038608,
        "_comment": "These percentages are tweaked so a regular praise+sourcecred allocation amounts to 7% of total rewards to quantfiers and 3% to the rewardboard."
      },
      "quantification_settings": {
        "number_of_quantifiers_per_praise_receiver": 3,
        "praise_quantify_allowed_values": [
          0, 1, 3, 5, 8, 13, 21, 34, 55, 89, 144
        ],
        "praise_quantify_receiver_prseudonyms": False,
        "praise_quantify_duplicate_praise_valuation": 0.1
      }
}

# Rewards Analytics and Distribution Dashboard
This goal of this notebook is to offer an easy way to process the outputs of the praise and sourcecred reward systems, perform an analysis of the results and calculate the token reward distribution. It uses mock data and should be considered a work-in-progress. 

### Imports
First, we import the relevant libraries, get the Data and set how many tokens we want to distribute

In [ ]:

import os
import sys

import math
import pandas as pd 
import numpy as np 

import scrapbook as sb


Now that we have selected the files, we can import them for processing. We can also set the total amount of tokens we want to distribute this period. We also set the name we want our output file to have.

Tip: Now that the file paths are set, you can safely click on "Cell > Run all below" from here on the menu bar to execute everything :) 

### Parameters

Set the total amount of tokens we want to distribute this period. We also set the name we want our output file to have.

In [ ]:
PRAISE_DATA_PATH = input_files["praise_data"]
REWARD_BOARD_ADDRESSES_PATH = input_files["rewardboard_list"]

praise_data = pd.read_csv(PRAISE_DATA_PATH)

rewardboard_addresses = pd.read_csv(REWARD_BOARD_ADDRESSES_PATH)

In [ ]:
NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE = math.trunc( float(total_tokens_allocated) * token_reward_percentages["contributor_rewards"]*1000000) / 1000000
#NUMBER_OF_SOURCECRED_REWARD_TOKENS_TO_DISTRIBUTE = 1950
NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS = math.trunc(  float(total_tokens_allocated) * token_reward_percentages["quantifier_rewards"]*1000000) / 1000000
NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD = math.trunc(  float(total_tokens_allocated) * token_reward_percentages["rewardboard_rewards"]*1000000) / 1000000
NUMBER_OF_QUANTIFIERS_PER_PRAISE = params["quantification_settings"]["number_of_quantifiers_per_praise_receiver"]

OUTPUT_FILENAME = "rewards_round_01"

## Reward allocation

### Praise

This method allocates the praise rewards in a very straightforward way: It adds the value of all dished praised together, and then assigns to each user their % of the total.

In [ ]:
def calc_praise_rewards(praiseData, tokensToDistribute):
    #we discard all we don't need and and calculate the % worth of each praise
    #slimData = praiseData[['FROM USER ACCOUNT', 'TO USER ACCOUNT', 'AVG SCORE']].copy()
    totalPraisePoints = praiseData['AVG SCORE'].sum()

    praiseData['PERCENTAGE'] = praiseData['AVG SCORE']/totalPraisePoints
    praiseData['TOKEN TO RECEIVE'] = praiseData['PERCENTAGE'] * tokensToDistribute
    return praiseData

praise_distribution = calc_praise_rewards(praise_data.copy(), NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE)
praise_distribution.style


### SourceCred
For now Sourcecred does the distribution independently, but if this changed we would calculate the rewards in the same way.

In [ ]:
#def calc_sourcecred_rewards(sourcecredData, tokensToDistribute):
#    #we discard all we don't need and and calculate the % worth of each praise
#    slimData = sourcecredData[['IDENTITY', 'AMOUNT']].copy()
#    totalGrainPoints = slimData['AMOUNT'].sum()

#    slimData['PERCENTAGE'] = slimData['AMOUNT']/totalGrainPoints
#    slimData['TOKEN TO RECEIVE'] = slimData['PERCENTAGE'] * tokensToDistribute
 #   return slimData

#sourcecred_distribution = calc_sourcecred_rewards(sourcecred_data.copy(), NUMBER_OF_SOURCECRED_REWARD_TOKENS_TO_DISTRIBUTE)
#sourcecred_distribution = sourcecred_data.copy()

## Preparing and combining the Datasets

Now that we have both distributions, we can combine them into one table.
But before that, we need to prepare the data and clean it a bit. We also use the chance to generate a table which shows us how much praise each user received. We'll use it later in our analysis.

In [ ]:

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -removes the '#' and following from discord names
#  -Some renaming and dropping 
def prepare_praise(praise_data):

    #praise_data['TO USER ACCOUNT'] = (praise_data['TO USER ACCOUNT'].str.split('#', 1, expand=False).str[0]).str.lower()
    
    praise_data.rename(columns = {'TO USER ACCOUNT':'USER IDENTITY'}, inplace = True)
    praise_data.rename(columns = {'TO ETH ADDRESS':'USER ADDRESS'}, inplace = True)
    praise_data['USER ADDRESS'].fillna('MISSING USER ADDRESS', inplace=True)
    
    processed_praise = praise_data[['USER IDENTITY', 'USER ADDRESS', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    praise_by_user = praise_data[['USER IDENTITY', 'USER ADDRESS', 'AVG SCORE', 'PERCENTAGE', 'TOKEN TO RECEIVE']].copy().groupby(['USER IDENTITY', 'USER ADDRESS']).agg('sum').reset_index()
    
    return processed_praise, praise_by_user

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -Some renaming and dropping 
#  -changing percentages from 0 - 100 to 0.00-1.00
def prepare_sourcecred(sourcecred_data):
    
    sourcecred_data.rename(columns = {'%':'PERCENTAGE SOURCECRED'}, inplace = True)
    sourcecred_data.rename(columns = {'IDENTITY' : 'USER ADDRESS'}, inplace = True)
    
    sourcecred_data = sourcecred_data[['USER ADDRESS', 'PERCENTAGE SOURCECRED', 'TOKEN TO RECEIVE']]
    
    return sourcecred_data


processed_praise, praise_by_user = prepare_praise(praise_distribution.copy())
#processed_sourcecred = prepare_sourcecred(sourcecred_distribution.copy())
processed_praise.style


Let's also create a table which will let us focus on the quantifiers. It will show us what value each quantifier gave to each single praise item.

In [ ]:
#THE NEW CHANGES IN THE DATA FORMAT ARE GOING TO MESS WITH THIS. REVIEW!!!

def data_by_quantifier(praise_data):
    quant_only = pd.DataFrame()
    praise_data.drop(['DATE', 'TO USER ACCOUNT', 'TO ETH ADDRESS', 'FROM USER ACCOUNT', 'FROM ETH ADDRESS', 'REASON', 'SOURCE ID', 'SOURCE NAME', 'AVG SCORE'], axis=1, inplace=True)
    num_of_quants = NUMBER_OF_QUANTIFIERS_PER_PRAISE
    for i in range(num_of_quants):
        q_name =  str( 'QUANTIFIER '+ str(i+1) +' USERNAME' )
        q_addr =  str( 'QUANTIFIER '+ str(i+1) +' ETH ADDRESS')
        q_value = str('SCORE '+str(i+1) )
        buf = praise_data[['ID', q_name , q_addr, q_value ]].copy()
    
        buf.rename(columns={q_name: 'QUANT_ID', q_addr: 'QUANT_ADDRESS', q_value: 'QUANT_VALUE', 'ID':'PRAISE_ID'}, inplace=True)
        #print(buf)
        quant_only = quant_only.append(buf.copy(), ignore_index=True)

    columnsTitles = ['QUANT_ID', 'QUANT_ADDRESS', 'PRAISE_ID', 'QUANT_VALUE']
    quant_only.sort_values(['QUANT_ID', 'PRAISE_ID'], inplace=True)
    quant_only =  quant_only.reindex(columns=columnsTitles).reset_index(drop=True)
    return quant_only

quantifier_rating_table = data_by_quantifier(praise_data.copy())
quantifier_rating_table.style


Now, we will calculate the rewards for the Quantifiers and the Reward Board. This is fairly straightforward: we distribute the tokens allocated for quantification proportionally to the number of praises quantified, and give all rewardboard members an equal cut.


In [ ]:


quantifier_rewards = pd.DataFrame(quantifier_rating_table[['QUANT_ID','QUANT_ADDRESS']].value_counts().reset_index().copy())
quantifier_rewards.rename(columns={ quantifier_rewards.columns[2]: "NUMBER_OF_PRAISES" }, inplace = True)

total_praise_quantified = quantifier_rewards['NUMBER_OF_PRAISES'].sum()
quantifier_rewards['TOKEN TO RECEIVE'] = quantifier_rewards['NUMBER_OF_PRAISES'] / total_praise_quantified  * NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS


    
quantifier_rewards.style

In [ ]:
rewardboard_rewards = pd.DataFrame(rewardboard_addresses)
rewardboard_rewards['TOKEN TO RECEIVE'] = NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD / len(rewardboard_rewards.index)
rewardboard_rewards.style

Now we can merge them all into one table and save it, ready for distribution!

In [ ]:
#def prepare_total_data_chart(praise_rewards, sourcecred_rewards, quantifier_rewards, rewardboard_rewards):
def prepare_total_data_chart(praise_rewards, quantifier_rewards, rewardboard_rewards):
    
    praise_rewards = praise_rewards.copy()[['USER IDENTITY', 'USER ADDRESS', 'TOKEN TO RECEIVE']].rename(columns = {'TOKEN TO RECEIVE':'PRAISE_REWARD'})
    praise_rewards['USER ADDRESS'] = praise_rewards['USER ADDRESS'].str.lower()
    
    #sourcecred_rewards = sourcecred_rewards.copy()[['USER ADDRESS', 'TOKEN TO RECEIVE']].rename(columns = {'TOKEN TO RECEIVE':'SOURCECRED_REWARD'})
    #sourcecred_rewards['USER ADDRESS'] = sourcecred_rewards['USER ADDRESS'].str.lower()
    
    quantifier_rewards.rename(columns = {'QUANT_ADDRESS':'USER ADDRESS', 'QUANT_ID': 'USER IDENTITY', 'NUMBER_OF_PRAISES': 'NR_OF_PRAISES_QUANTIFIED', 'TOKEN TO RECEIVE':'QUANT_REWARD'}, inplace = True)
    quantifier_rewards['USER ADDRESS'] = quantifier_rewards['USER ADDRESS'].str.lower()
    
    rewardboard_rewards.rename(columns = {'ID':'USER ADDRESS', 'TOKEN TO RECEIVE': 'REWARDBOARD_REWARD'}, inplace = True)
    rewardboard_rewards['USER ADDRESS'] = rewardboard_rewards['USER ADDRESS'].str.lower()
    
    
    final_allocations = pd.merge(rewardboard_rewards, quantifier_rewards , on=['USER ADDRESS','USER ADDRESS'], how='outer')
    #final_allocations = pd.merge(final_allocations , sourcecred_rewards, on=['USER ADDRESS','USER ADDRESS'],how='outer')
    final_allocations = pd.merge(final_allocations, praise_rewards, left_on=['USER ADDRESS'], right_on=['USER ADDRESS'], how='outer')
    
    #now we can merge the IDs, replacing any missing values
    final_allocations['USER IDENTITY_x']= final_allocations['USER IDENTITY_x'].combine_first(final_allocations['USER IDENTITY_y'])
    final_allocations.rename(columns = {'USER IDENTITY_x': 'USER IDENTITY'},  inplace = True)
    final_allocations.drop('USER IDENTITY_y', axis=1, inplace=True)
    
    
    final_allocations['USER IDENTITY'].fillna('missing username', inplace = True)
    final_allocations.fillna(0, inplace = True)
    #final_allocations['TOTAL TO RECEIVE'] = final_allocations['PRAISE_REWARD'] + final_allocations['SOURCECRED_REWARD'] + final_allocations['QUANT_REWARD'] + final_allocations['REWARDBOARD_REWARD']
    final_allocations['TOTAL TO RECEIVE'] = final_allocations['PRAISE_REWARD'] + final_allocations['QUANT_REWARD'] + final_allocations['REWARDBOARD_REWARD']
   
    
    final_allocations = final_allocations.sort_values(by= 'TOTAL TO RECEIVE', ascending  = False).reset_index(drop=True)
    
    #put the columns into the desired order
    #final_allocations = final_allocations[['USER IDENTITY', 'USER ADDRESS', 'PRAISE_REWARD', 'SOURCECRED_REWARD', 'QUANT_REWARD','NR_OF_PRAISES_QUANTIFIED', 'REWARDBOARD_REWARD', 'TOTAL TO RECEIVE']]
    final_allocations = final_allocations[['USER IDENTITY', 'USER ADDRESS', 'PRAISE_REWARD', 'QUANT_REWARD','NR_OF_PRAISES_QUANTIFIED', 'REWARDBOARD_REWARD', 'TOTAL TO RECEIVE']]
    
    
    return final_allocations

#final_token_allocations = prepare_total_data_chart(praise_by_user.copy(), processed_sourcecred.copy(), quantifier_rewards.copy(), rewardboard_rewards.copy())

final_token_allocations = prepare_total_data_chart(praise_by_user.copy(), quantifier_rewards.copy(), rewardboard_rewards.copy())
final_token_allocations.style


### "Glue" relevant DataFrames to send to analysis

In [ ]:
sb.glue("final_token_allocations", final_token_allocations, 'pandas')
sb.glue("rewardboard_rewards", rewardboard_rewards, 'pandas')
sb.glue("quantifier_rewards", quantifier_rewards, 'pandas')
sb.glue("quantifier_rating_table", quantifier_rating_table, 'pandas')

sb.glue("processed_praise", processed_praise, 'pandas')
sb.glue("praise_by_user", praise_by_user, 'pandas')
#sb.glue("processed_sourcecred", processed_sourcecred, 'pandas')

#sb.glue("sourcecred_distribution", sourcecred_distribution, 'pandas')
sb.glue("praise_distribution", praise_distribution, 'pandas')
sb.glue("quantifiers_per_praise", quantification_settings["number_of_quantifiers_per_praise_receiver"])

### Save the distribution files

In [ ]:
#TO DO:
#save final allocation to file called "final_praise_token_allocation.csv"
#save a disperse compatible "praise_disperse_distribution.csv" (does it need token address?)
#save the extended praise DataFrame (the one with every praise and its weight) "extended_praise_data.csv"

In [ ]:
final_allocation_csv = final_token_allocations.to_csv(index=False)
with open('final_praise_token_allocation.csv', 'w') as f:
    f.write(final_allocation_csv)

In [ ]:
final_alloc_disperse = final_token_allocations[['USER ADDRESS', 'TOTAL TO RECEIVE']].to_csv(sep=' ', index=False, header=False)
with open('praise_disperse_distribution.csv', 'w') as f:
    f.write(final_alloc_disperse)

In [ ]:
praise_reward_export = praise_distribution.to_csv(index=False)
with open('extended_praise_data.csv', 'w') as f:
    f.write(praise_reward_export)